In [1]:
import os
from home_monitoring_display.utils import load_config

In [2]:
conf = load_config(os.path.abspath('../conf/streaming_config.yaml'))

In [271]:
client_conf = conf["influxdb_connectors"]["homemonitor"]

In [272]:
measurement = "bme688"
field = "temperature"
start = "7d"
stop = "now()"

In [291]:
client_conf = conf["influxdb_connectors"]["pi"]

In [292]:
measurement = "teleinfo"
field = "PAPP"
start = "7d"
stop = "now()"

# Test influxdb_client

In [6]:
import influxdb_client as idb

In [7]:
client1 = idb.InfluxDBClient(
                url=f"http://{client_conf['host']}:8086",
                token=f"{client_conf['username']}:{client_conf['password']}",
                org="-",
                timeout=200_000
            )

query_api = client1.query_api()

time for bme688.temperature :

In [82]:
%%timeit
query1 = f"""from(bucket: "{client_conf['database']}/autogen")
                    |> range(start: {start}, stop: {stop})
                    |> filter(fn: (r) => r._measurement == "{measurement}")
                    |> filter(fn: (r) => r._field == "{field}")
                    |> pivot(rowKey:["_time"], columnKey: ["_field"], valueColumn: "_value")
                    |> keep(columns: ["_time", "{field}"])"""
                    
df_query1 = query_api.query_data_frame(query1, data_frame_index=["_time"])

2.41 s ± 147 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


time for teleinfo.PAPP : 

In [ ]:
%%timeit
query1 = f"""from(bucket: "{client_conf['database']}/autogen")
                    |> range(start: {start}, stop: {stop})
                    |> filter(fn: (r) => r._measurement == "{measurement}")
                    |> filter(fn: (r) => r._field == "{field}")
                    |> pivot(rowKey:["_time"], columnKey: ["_field"], valueColumn: "_value")
                    |> keep(columns: ["_time", "{field}"])"""
                    
df_query1 = query_api.query_data_frame(query1, data_frame_index=["_time"])

In [9]:
%%time
query1 = f"""from(bucket: "{client_conf['database']}/autogen")
                    |> range(start: {start}, stop: {stop})
                    |> filter(fn: (r) => r._measurement == "{measurement}")
                    |> filter(fn: (r) => r._field == "{field}")
                    |> pivot(rowKey:["_time"], columnKey: ["_field"], valueColumn: "_value")
                    |> keep(columns: ["_time", "{field}"])"""
                    
df_query1 = query_api.query_data_frame(query1, data_frame_index=["_time"])

CPU times: user 40.1 s, sys: 626 ms, total: 40.7 s
Wall time: 1min 2s


In [26]:
%%time
query1 = f"""from(bucket: "{client_conf['database']}/autogen")
                    |> range(start: {start}, stop: {stop})
                    |> filter(fn: (r) => r._measurement == "{measurement}" and r._field == "{field}")
                    |> keep(columns: ["_time", "_value"])
                    |> window(every: 5m)
                    |> mean()"""
                    
result_query1 = query_api.query(query1)

ApiException: (500)
Reason: Internal Server Error
HTTP response headers: HTTPHeaderDict({'Content-Type': 'text/csv; charset=utf-8', 'Request-Id': '99fabd91-da2f-11ed-a866-b827eb20b9db', 'X-Influxdb-Build': 'OSS', 'X-Influxdb-Error': 'panic: runtime error: invalid memory address or nil pointer dereference', 'X-Influxdb-Version': '1.8.10', 'X-Request-Id': '99fabd91-da2f-11ed-a866-b827eb20b9db', 'Date': 'Thu, 13 Apr 2023 19:17:23 GMT', 'Transfer-Encoding': 'chunked'})
HTTP response body: b'{"error":"panic: runtime error: invalid memory address or nil pointer dereference"}\n'


In [ ]:
%%time
query1 = f"""from(bucket: "{client_conf['database']}/autogen")
                    |> range(start: {start}, stop: {stop})
                    |> filter(fn: (r) => r._measurement == "{measurement}")
                    |> filter(fn: (r) => r._field == "{field}")
                    |> pivot(rowKey:["_time"], columnKey: ["_field"], valueColumn: "_value")
                    |> keep(columns: ["_time", "{field}"])
                    |> aggregateWindow(every: 30s, fn: mean)"""
                    
df_query1 = query_api.query_data_frame(query1, data_frame_index=["_time"])

In [18]:
df_query1

,result,table,PAPP
_time,,,
2023-03-30 16:43:12.488021+00:00,_result,0,30
2023-03-30 16:43:13.916776+00:00,_result,0,30
2023-03-30 16:43:15.425432+00:00,_result,0,30
2023-03-30 16:43:16.916091+00:00,_result,0,30
2023-03-30 16:43:18.403792+00:00,_result,0,30
...,...,...,...
2023-04-06 16:43:03.725629+00:00,_result,0,130
2023-04-06 16:43:05.227356+00:00,_result,0,140
2023-04-06 16:43:06.717376+00:00,_result,0,130


# Test influxdb

In [91]:
from influxdb import DataFrameClient, InfluxDBClient

In [ ]:
client 

In [293]:
client2 = DataFrameClient(client_conf["host"], 8086, client_conf["username"], client_conf["password"], database=client_conf["database"], timeout=200)

In [97]:
client2.get_list_measurements()

[{'name': 'bme688'}, {'name': 'openweather_current_weather'}]

In [274]:
query2 = f"""SELECT {field} FROM {measurement}
            WHERE time > now() - {start} AND time < {stop} """

In [275]:
query2

'SELECT temperature FROM bme688\n            WHERE time > now() - 7d AND time < now() '

time for bme688.temperature : 

In [81]:
%%timeit
df_query2 = client2.query(query2)[measurement]

677 ms ± 172 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


time for teleinfo.PAPP : 

In [48]:
%%timeit
df_query2 = client2.query(query2)[measurement]

57.3 s ± 3.8 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [276]:
%%time
df_query2 = client2.query(query2)[measurement]

CPU times: user 97.2 ms, sys: 26.2 ms, total: 123 ms
Wall time: 775 ms


In [106]:
df_query2

,temperature
2023-04-07 20:05:27.859447+00:00,18.28
2023-04-07 20:05:57.862599+00:00,18.29
2023-04-07 20:06:27.867723+00:00,18.30
2023-04-07 20:06:57.867264+00:00,18.32
2023-04-07 20:07:27.868901+00:00,18.34
...,...
2023-04-14 20:02:54.150653+00:00,17.64
2023-04-14 20:03:24.151740+00:00,17.64
2023-04-14 20:03:54.154757+00:00,17.64
2023-04-14 20:04:24.155225+00:00,17.63


In [70]:
query2 = f"""SELECT mean({field}) FROM {client_conf["database"]}."autogen"."{measurement}"
            WHERE time > now() {start} AND time < {stop} """

In [71]:
result = client2.query(query2)

In [50]:
query3 = f"""SELECT mean({field}) as "mean" FROM {client_conf["database"]}.autogen.{measurement}
            WHERE time > {start} AND time < {stop} GROUP BY time(30s) FILL(null)"""

In [62]:
%%time
df_query3 = client2.query(query3)[measurement]

CPU times: user 5.46 ms, sys: 1.79 ms, total: 7.25 ms
Wall time: 83.4 ms


In [169]:
query4 = f"SELECT * FROM {client_conf['database']}.autogen.{measurement} LIMIT 1"

# Get schema

In [277]:
query4 = f"SELECT first(*) FROM {client_conf['database']}.autogen.{measurement}"

In [278]:
df_query = client2.query(query4)[measurement]

In [281]:
df_query.index[0]

Timestamp('1970-01-01 00:00:00+0000', tz='UTC')

In [286]:
query4 = f"SHOW FIELD KEYS"

In [287]:
fields_set = client2.query(query4)

In [288]:
fields_set

ResultSet({'('bme688', None)': [{'fieldKey': 'gas_resistance', 'fieldType': 'float'}, {'fieldKey': 'heat_stable', 'fieldType': 'boolean'}, {'fieldKey': 'humidity', 'fieldType': 'float'}, {'fieldKey': 'pressure', 'fieldType': 'float'}, {'fieldKey': 'status', 'fieldType': 'integer'}, {'fieldKey': 'temperature', 'fieldType': 'float'}], '('openweather_current_weather', None)': [{'fieldKey': 'cloud_cover', 'fieldType': 'integer'}, {'fieldKey': 'humidity', 'fieldType': 'integer'}, {'fieldKey': 'pressure', 'fieldType': 'integer'}, {'fieldKey': 'temp', 'fieldType': 'float'}, {'fieldKey': 'temp_feels_like', 'fieldType': 'float'}, {'fieldKey': 'weather_category', 'fieldType': 'string'}, {'fieldKey': 'weather_description', 'fieldType': 'string'}, {'fieldKey': 'wind_direction', 'fieldType': 'integer'}, {'fieldKey': 'wind_gust', 'fieldType': 'float'}, {'fieldKey': 'wind_speed', 'fieldType': 'float'}]})

In [294]:
query4 = f"SHOW FIELD KEYS"
fields_set = client2.query(query4)

schema = {}
for measurement, fields_meas in fields_set.items():
    fields = {field_dict['fieldKey']: field_dict['fieldType']
              for field_dict in fields_meas }
    first_field = list(fields.keys())[0]
     
    query_first = f"SELECT first({first_field}) FROM {measurement[0]}"
    query_last = f"SELECT last({first_field}) FROM {measurement[0]}"
    
    schema[measurement[0]] = {
        "fields": fields,
        "first_date": client2.query(query_first)[measurement[0]].index[0],
        "last_date": client2.query(query_last)[measurement[0]].index[0],
    }
    

In [295]:
schema

{'sensorhub': {'fields': {'Brightness': 'integer',
   'HumanDetected': 'boolean',
   'Humidity': 'integer',
   'Pressure': 'float',
   'TempBaromter': 'integer',
   'TempOffchip': 'integer',
   'TempOnChip': 'integer'},
  'first_date': Timestamp('2023-03-15 18:24:38.354230+0000', tz='UTC'),
  'last_date': Timestamp('2023-04-16 21:36:28.119821+0000', tz='UTC')},
 'teleinfo': {'fields': {'BASE': 'integer',
   'HHPHC': 'string',
   'IINST': 'integer',
   'IMAX': 'integer',
   'PAPP': 'integer'},
  'first_date': Timestamp('2023-03-15 18:23:41.959361+0000', tz='UTC'),
  'last_date': Timestamp('2023-04-16 21:36:57.823737+0000', tz='UTC')}}

In [222]:
for database, fields_gen in fields_set.items():
    for field_dict in fields_gen: 
        print(field_dict[field])

TypeError: 'dict_values' object is not subscriptable

In [214]:
test_list = ['bonjour', 'connard']    

In [216]:
dict.fromkeys(*test_list)

{'b': 'connard',
 'o': 'connard',
 'n': 'connard',
 'j': 'connard',
 'u': 'connard',
 'r': 'connard'}

In [213]:
dict.fromkeys(['bonjour'], 'connard')

{'bonjour': 'connard'}

In [206]:
schema

{'sensorhub': {'fields': {dict_values(['Brightness', 'integer']),
   dict_values(['HumanDetected', 'boolean']),
   dict_values(['Humidity', 'integer']),
   dict_values(['Pressure', 'float']),
   dict_values(['TempBaromter', 'integer']),
   dict_values(['TempOffchip', 'integer']),
   dict_values(['TempOnChip', 'integer'])}},
 'teleinfo': {'fields': {dict_values(['BASE', 'integer']),
   dict_values(['HHPHC', 'string']),
   dict_values(['IINST', 'integer']),
   dict_values(['IMAX', 'integer']),
   dict_values(['PAPP', 'integer'])}}}

In [ ]:
df_query = 

In [191]:
for field in df_query['teleinfo']:
    print(field)

{'fieldKey': 'BASE', 'fieldType': 'integer'}
{'fieldKey': 'HHPHC', 'fieldType': 'string'}
{'fieldKey': 'IINST', 'fieldType': 'integer'}
{'fieldKey': 'IMAX', 'fieldType': 'integer'}
{'fieldKey': 'PAPP', 'fieldType': 'integer'}


In [173]:
df_query.dtypes

BASE      int64
HHPHC    object
IINST     int64
IMAX      int64
PAPP      int64
dtype: object

In [167]:
df_query

,first
2023-03-15 18:23:41.959361+00:00,8172459


In [148]:
client2.get_list_series(database=client_conf['database'], measurement=measurement)

['teleinfo']

In [168]:
df_query

,first
2023-03-15 18:23:41.959361+00:00,8172459


# Test equality

In [77]:
(df_query["temperature"].reset_index(drop=True)== df_query2["temperature"].reset_index(drop=True)).sum()

20159

In [72]:
df_query.rename_axis(None, axis=0)["temperature"]

2023-03-29 19:36:53.013039+00:00    18.44
2023-03-29 19:37:23.016486+00:00    18.44
2023-03-29 19:37:53.016595+00:00    18.44
2023-03-29 19:38:23.018836+00:00    18.44
2023-03-29 19:38:53.019718+00:00    18.44
                                    ...  
2023-04-05 19:34:20.299559+00:00    17.45
2023-04-05 19:34:50.299772+00:00    17.46
2023-04-05 19:35:20.300499+00:00    17.46
2023-04-05 19:35:50.303355+00:00    17.46
2023-04-05 19:36:20.302933+00:00    17.47
Name: temperature, Length: 20159, dtype: float64

In [65]:
df_query2["temperature"]

2023-03-29 19:36:53.013039+00:00    18.44
2023-03-29 19:37:23.016487+00:00    18.44
2023-03-29 19:37:53.016596+00:00    18.44
2023-03-29 19:38:23.018837+00:00    18.44
2023-03-29 19:38:53.019718+00:00    18.44
                                    ...  
2023-04-05 19:34:20.299559+00:00    17.45
2023-04-05 19:34:50.299773+00:00    17.46
2023-04-05 19:35:20.300500+00:00    17.46
2023-04-05 19:35:50.303356+00:00    17.46
2023-04-05 19:36:20.302933+00:00    17.47
Name: temperature, Length: 20159, dtype: float64

In [53]:
df_query2

,temperature
2023-03-29 19:36:53.013039+00:00,18.44
2023-03-29 19:37:23.016487+00:00,18.44
2023-03-29 19:37:53.016596+00:00,18.44
2023-03-29 19:38:23.018837+00:00,18.44
2023-03-29 19:38:53.019718+00:00,18.44
...,...
2023-04-05 19:34:20.299559+00:00,17.45
2023-04-05 19:34:50.299773+00:00,17.46
2023-04-05 19:35:20.300500+00:00,17.46
2023-04-05 19:35:50.303356+00:00,17.46


# Test new InfluxDBConnector

In [296]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [297]:
from home_monitoring_display.query_influxdb import InfluxDBConnector

In [298]:
influxdb_connector = InfluxDBConnector(**client_conf)

In [299]:
influxdb_connector.get_schema()

{'sensorhub': {'fields': {'Brightness': 'integer',
   'HumanDetected': 'boolean',
   'Humidity': 'integer',
   'Pressure': 'float',
   'TempBaromter': 'integer',
   'TempOffchip': 'integer',
   'TempOnChip': 'integer'},
  'first_date': Timestamp('2023-03-15 18:24:38.354230+0000', tz='UTC'),
  'last_date': Timestamp('2023-04-16 21:38:28.079489+0000', tz='UTC')},
 'teleinfo': {'fields': {'BASE': 'integer',
   'HHPHC': 'string',
   'IINST': 'integer',
   'IMAX': 'integer',
   'PAPP': 'integer'},
  'first_date': Timestamp('2023-03-15 18:23:41.959361+0000', tz='UTC'),
  'last_date': Timestamp('2023-04-16 21:38:43.676188+0000', tz='UTC')}}

In [83]:
start = '7d'

In [84]:
df_query = influxdb_connector.query_field(measurement, field, start, stop)

In [86]:
df_query = influxdb_connector.query_field(measurement, field, start, stop, groupby_interval="30s")

In [73]:
mean = influxdb_connector.query_mean_field(measurement, field, start, stop)

                                  temperature
2023-04-07 18:57:36.948831+00:00    17.647745


In [81]:
df_query = influxdb_connector.query_field(measurement, field, "10m", stop)

In [75]:
mean

17.647744927823922

In [45]:
print(query2)

SELECT temperature FROM homemonitor.autogen.bme688
            WHERE time > now() -7d AND time < now() 


In [47]:
df_query

,_time,temperature
0,2023-04-07 20:48:57.651654+02:00,18.29
1,2023-04-07 20:49:27.656088+02:00,18.28
2,2023-04-07 20:49:57.654600+02:00,18.29
3,2023-04-07 20:50:27.657079+02:00,18.29
4,2023-04-07 20:50:57.658990+02:00,18.29
...,...,...
20155,2023-04-14 20:46:53.936607+02:00,17.76
20156,2023-04-14 20:47:23.950885+02:00,17.75
20157,2023-04-14 20:47:53.951299+02:00,17.74
20158,2023-04-14 20:48:23.953750+02:00,17.73
